In [ ]:
fileName = f'emap_msg.txt'
import os
curPath = os.getcwd()
filePath = os.path.join(curPath, fileName)
os.path.exists(filePath)
filePath

In [ ]:
fileLine = []

LOG_FILE_NAME = 'msg_manager.cpp'

with open(filePath, 'r') as f:
    for line in f:
        if LOG_FILE_NAME in line:
            fileLine.append(line.strip())

filterLogWithMsgMgrFileName = f'emap_msg_mgr.txt'
filePath = os.path.join(curPath, filterLogWithMsgMgrFileName)

with open(filePath, 'w') as f:
    for line in fileLine:
        f.write(line + '\n')

In [ ]:
with open(filePath, 'r') as f:
    fileLine = []
    for line in f:
        fileLine.append(line)
fileLine[-10:-1]

In [ ]:
import re

recvMsgDict = {}
sendMsgDict = {}

RECV_MSG_PREFIX = 'Recv msg'
SEND_MSG_PREFIX = 'Send msg'

#  >>([a-z|A-Z]{4} msg) \[ \d{2}\]: (.*) :: (.*)

pattern = r'.*?\[(\d{2}:\d{2}:\d{2}\.\d{3})\]: >>([a-z|A-Z]{4} msg)\[ \d*?\]: (.*) :: (.*)'
for line in fileLine:
    msg = re.findall(pattern=pattern, string=line)
    if len(msg) != 0:
        time, msgPrefix, msgHead, msgTail = msg[0]
        if msgPrefix == RECV_MSG_PREFIX:
            recvMsgDict[ ( RECV_MSG_PREFIX, time ) ] = [ msgHead, msgTail ]
        elif msgPrefix == SEND_MSG_PREFIX:
            sendMsgDict[ ( SEND_MSG_PREFIX, time ) ] = [ msgHead, msgTail ]
        else:
            exit(-1)

recvMsgDict, sendMsgDict

In [ ]:
msgNo = 1
recvMsgItem = recvMsgDict[ ( RECV_MSG_PREFIX, '21:43:30.574' ) ]
recvMsgItem

In [ ]:
pattern = r'[\d|A-F|a-f]{2}'
splitMsgList = re.findall(pattern=pattern, string=recvMsgItem[0])
srcModuleId = [ splitMsgList[0], splitMsgList[1] ]
srcModuleId.reverse()

destModuleId = [ splitMsgList[2], splitMsgList[3] ]
destModuleId.reverse()

eval('0x' + ''.join(srcModuleId)), eval('0x' + ''.join(destModuleId))

In [ ]:
from datetime import datetime

def convert_time_string(time_string):
    # 定义时间字符串的格式
    time_format = "%H:%M:%S.%f"
    # 将时间字符串解析为datetime对象
    time_obj = datetime.strptime(time_string, time_format)
    return time_obj

from typing import Dict, List, Tuple

def get_time(msgDict: Dict[Tuple, List], timeRange: Tuple[ str, str ]):
    startTime = convert_time_string(timeRange[0])
    endTime = convert_time_string(timeRange[1])
    timeInRangeList = []
    msgSeqList = []
    for msg in msgDict:
        time = msg[1]
        timeObj = convert_time_string(time)
        msgItem = msgDict[msg]

        pattern = r'[\d|A-F|a-f]{2}'
        msgHead = re.findall(pattern, msgItem[0])

        srcModuleId = [ msgHead[0], msgHead[1] ]
        destModuleId = [ msgHead[2], msgHead[3] ]
        msgTypeId = [ msgHead[4], msgHead[5] ]
        msgSeq = [ msgHead[6], msgHead[7] ]

        srcModuleId.reverse()
        destModuleId.reverse()
        msgTypeId.reverse()
        msgSeq.reverse()

        srcModuleId = '0x' + ''.join(srcModuleId)
        destModuleId = '0x' + ''.join(destModuleId)
        msgTypeId = '0x' + ''.join(msgTypeId)
        msgSeq = '0x' + ''.join(msgSeq)

        # print(srcModuleId, destModuleId, msgTypeId)

        if startTime <= timeObj < endTime:
            timeInRangeList.append([ timeObj, "srcModuleId:", f"{eval(srcModuleId):04}", "destModuleId:", f"{eval(destModuleId):04}", "msgTypeId:", msgTypeId, "msgSeq:", msgSeq ])
            msgSeqList.append(msgSeq)
    return timeInRangeList, msgSeqList


recvTimeInRangeList, recvMsgSeqList = get_time(recvMsgDict, ( '21:43:30.000', '21:44:01.000' ))
recvTimeInRangeList

In [ ]:
sendTimeInRangeList, sendMsgSeqList = get_time(sendMsgDict, ( '21:43:50.000', '21:43:59.000' ))
sendTimeInRangeList

In [ ]:
# 查找重复消息序号
def find_duplicates(lst):
    frequency = {}
    duplicates = []

    for item in lst:
        if item in frequency:
            frequency[item] += 1
        else:
            frequency[item] = 1

    for key, value in frequency.items():
        if value > 1:
            duplicates.append(key)

    return duplicates

dup = find_duplicates(recvMsgSeqList)
dup

In [ ]:
# 按消息序号查找收发间隔
def traversal_send_receive_interval(recvMsgList, sendMsgList):
    for recvMsg in recvMsgList:
        msgSeq = recvMsg[6]
        srcModuleId = recvMsg[1]
        destModuleId = recvMsg[2]
        if srcModuleId == destModuleId:
            continue
        if msgSeq == '0x0000':
            continue
        time = recvMsg[0]
        for sendMsg in sendMsgList:
            if sendMsg[6] == msgSeq:
                if sendMsg[0] < time:
                    continue
                timeInterval = sendMsg[0] - time
                if timeInterval > convert_time_string('0:00:00.200000') - convert_time_string('0:00:00.000000'):
                    print(recvMsg[1:], ': ', timeInterval)

traversal_send_receive_interval(recvTimeInRangeList, sendTimeInRangeList)